In [1]:
from keras.datasets import mnist
import numpy as np
(X_train, y_train), (X_test, y_test) = mnist.load_data()
y_train, y_test = np.eye(10)[y_train.astype(int)], np.eye(10)[y_test.astype(int)]
X_train = X_train.reshape(-1, 28*28)
X_train.shape
X_test = X_test.reshape(-1, 28*28)
X_test.shape
n_in = len(X_train[0])
n_in
n_hidden = 200
n_out = 10

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

In [5]:
import tensorflow as tf
from sklearn.utils import shuffle
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

x = tf.placeholder(tf.float32, shape=[None, n_in])
t = tf.placeholder(tf.float32, shape=[None, n_out])
keep_prob = tf.placeholder(tf.float32) # 드랍아웃하지 않을 확률

W0 = tf.Variable(tf.truncated_normal([n_in, n_hidden], stddev=0.01))
b0 = tf.Variable(tf.zeros([n_hidden]))
h0 = tf.nn.relu(tf.matmul(x, W0) + b0)
h0_drop = tf.nn.dropout(h0, keep_prob)

W1 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b1 = tf.Variable(tf.zeros([n_hidden]))
h1 = tf.nn.relu(tf.matmul(h0_drop, W1)+b1)
h1_drop = tf.nn.dropout(h1, keep_prob)

W2 = tf.Variable(tf.truncated_normal([n_hidden, n_out], stddev=0.01))
b2 = tf.Variable(tf.zeros([n_out]))
y = tf.nn.softmax(tf.matmul(h1_drop, W2) + b2)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [13]:
cross_entropy = -tf.reduce_sum(t*tf.log(y) + (1-t)*tf.log(1-y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [14]:
# 학습
batch_size = 100
n_batches = 60000 // 100
for epoch in range(30):
    X_, Y_ = shuffle(X_train, y_train)
    
    for i in range(n_batches):
        start = i*batch_size
        end = start+batch_size
        
        sess.run(train_step, feed_dict={
            x:X_[start:end], t:Y_[start:end], keep_prob:0.5 #여기에서 keep_prob설정해주기
        })

In [16]:
correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

Instructions for updating:
Use `tf.cast` instead.


In [17]:
# Accuracy 정의하기
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
accuracy_rate = accuracy.eval(session=sess, feed_dict={
    x:X_test, t:y_test, keep_prob: 1.0
})

In [19]:
accuracy_rate

0.9

### Keras로 구현해보기

In [20]:
from keras.layers.core import Dropout

In [21]:
model = Sequential()
model.add(Dense(n_hidden, input_dim=n_in))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_hidden))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_hidden))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_out))
model.add(Activation('softmax'))

In [24]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01),metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 133us/sample - loss: 1.1555 - acc: 0.5968
Epoch 2/10
60000/60000 [==============================] - 8s 131us/sample - loss: 0.9816 - acc: 0.6630
Epoch 3/10
60000/60000 [==============================] - 8s 132us/sample - loss: 0.9316 - acc: 0.6826
Epoch 4/10
60000/60000 [==============================] - 8s 132us/sample - loss: 0.8732 - acc: 0.7070
Epoch 5/10
60000/60000 [==============================] - 8s 132us/sample - loss: 0.8490 - acc: 0.7164
Epoch 6/10
60000/60000 [==============================] - 8s 132us/sample - loss: 0.8068 - acc: 0.7348
Epoch 7/10
60000/60000 [==============================] - 8s 132us/sample - loss: 0.7934 - acc: 0.7395
Epoch 8/10
60000/60000 [==============================] - 8s 133us/sample - loss: 0.7515 - acc: 0.7533
Epoch 9/10
60000/60000 [==============================] - 8s 133us/sample - loss: 0.7248 - acc: 0.7644
Epoch 10/10
60000/60000 [=========================